# BinanceBot - Kaggle Training Notebook

**Strategy:** GodStra  
**Epochs:** 100 per timeframe  
**Timeframes:** 4h, 12h, 1d  
**Total Runtime:** ~21-22 hours

## Workflow:
1. Install BinanceBot from GitHub
2. Download historical data
3. Run Hyperopt (optimize parameters)
4. Run Backtest (test on unseen data)
5. Save and download results

## Prerequisites:
- Kaggle notebook with internet enabled
- GPU optional (CPU works fine)

In [ ]:
# CELL 1: Install BinanceBot from GitHub (Test Repository)
import subprocess
import sys

# Change this URL when ready to use main repo
GITHUB_REPO = "https://github.com/NDuc1602/binancebot.git"

print(f"Installing BinanceBot from: {GITHUB_REPO}")
print("This will take 3-5 minutes...")

# Install directly from GitHub repo
result = subprocess.run([
    sys.executable, "-m", "pip", "install", 
    "-q",
    f"git+{GITHUB_REPO}"
], capture_output=True, text=True)

if result.returncode == 0:
    print("✓ Installation successful!")
else:
    print("⚠ Installation completed with warnings (normal on Kaggle)")
    if "Successfully installed" in result.stdout or "Successfully installed" in result.stderr:
        print("✓ BinanceBot was installed despite warnings")

# Install ta library (required for GodStra strategy)
print("\nInstalling ta library for GodStra strategy...")
result_ta = subprocess.run([
    sys.executable, "-m", "pip", "install", "-q", "ta"
], capture_output=True, text=True)

if result_ta.returncode == 0:
    print("✓ ta library installed")
else:
    print("⚠ ta library installation had warnings (may still work)")

# Verify installation
print("\nVerifying BinanceBot...")
result = subprocess.run(['binancebot', '--version'], capture_output=True, text=True)
if result.returncode == 0:
    print(f"✓ BinanceBot {result.stdout.strip()}")
    print("✓ Ready to start!")
else:
    print("✗ Installation verification failed")
    print("Trying alternative check...")
    result2 = subprocess.run([sys.executable, "-m", "binancebot", "--version"], capture_output=True, text=True)
    if result2.returncode == 0:
        print(f"✓ BinanceBot {result2.stdout.strip()}")
        print("✓ Ready (use 'python -m binancebot' instead of 'binancebot')")
    else:
        raise RuntimeError("BinanceBot installation failed")

In [ ]:
# CELL 2: Import Libraries
import subprocess
import json
import shutil
from pathlib import Path
from datetime import datetime

print("✓ Libraries imported")

In [ ]:
# CELL 3: Configuration
CONFIG = {
    'strategy': 'GodStra',
    'timeframes': ['4h', '12h', '1d'],
    'pairs': ['BTC/USDT', 'ETH/USDT', 'BNB/USDT'],
    'epochs': 10,  # Start with 10 for validation, then change to 100 for full run
    'loss_function': 'SharpeHyperOptLoss',
    'hyperopt_timerange': '20200101-20231231',  # Train: 2020-2023 (4 years)
    'backtest_timerange': '20240101-20241122',  # Test: 2024-01-01 to today (11 months)
}

# Paths
WORK_DIR = Path('/kaggle/working')
USER_DATA = WORK_DIR / 'user_data'
CONFIG_PATH = WORK_DIR / 'config.json'

print("Configuration:")
print(f"  Strategy: {CONFIG['strategy']}")
print(f"  Timeframes: {CONFIG['timeframes']}")
print(f"  Epochs: {CONFIG['epochs']}")
print(f"  Pairs: {CONFIG['pairs']}")
print(f"  Total runs: {len(CONFIG['timeframes'])}")
print(f"  Train period: 2020-2023 (4 years)")
print(f"  Test period: 2024-now (11 months)")
if CONFIG['epochs'] == 10:
    print(f"  Estimated time: ~13 min per timeframe = ~40 min total (VALIDATION MODE)")
else:
    print(f"  Estimated time: ~7h per timeframe = ~21h total (FULL TRAINING)")

In [ ]:
# CELL 4: Create Config File
config_data = {
    "max_open_trades": 3,
    "stake_currency": "USDT",
    "stake_amount": 30,
    "dry_run": True,
    "timeframe": "4h",
    "trading_mode": "spot",  # SPOT trading only
    "margin_mode": "",  # Not applicable for spot
    "exchange": {
        "name": "binance",
        "pair_whitelist": CONFIG['pairs'],
        "pair_blacklist": [],
        # Disable API calls - we're using offline data only
        "skip_pair_validation": True,
        "skip_open_order_validation": True
    },
    "pairlists": [{"method": "StaticPairList"}],
    "entry_pricing": {
        "price_side": "same",
        "use_order_book": True,
        "order_book_top": 1,
        "check_depth_of_market": {
            "enabled": False,
            "bids_to_ask_delta": 1
        }
    },
    "exit_pricing": {
        "price_side": "same",
        "use_order_book": True,
        "order_book_top": 1
    }
}

CONFIG_PATH.write_text(json.dumps(config_data, indent=2))
print(f"✓ Config created: {CONFIG_PATH}")
print(f"  Trading mode: SPOT (buy/sell only, no short)")
print(f"  Exchange validation: DISABLED (offline backtesting mode)")

In [ ]:
# CELL 5: Create Strategy File
strategy_code = '''
# GodStra Strategy
# Author: @Mablue (Masoud Azizi)
# github: https://github.com/mablue/
# IMPORTANT: INSTALL TA BEFORE RUN (pip install ta)
# Modified for SPOT trading only (buy/sell, no short)

from functools import reduce
import binancebot.vendor.qtpylib.indicators as qtpylib
import numpy as np
from pandas import DataFrame
from binancebot.strategy import IStrategy, CategoricalParameter, IntParameter, DecimalParameter
from ta import add_all_ta_features
from ta.utils import dropna

class GodStra(IStrategy):
    INTERFACE_VERSION: int = 3
    
    # SPOT trading only - no short positions
    can_short = False
    
    # Hyperopt parameters for buy signals
    buy_oper_0 = CategoricalParameter([">", "=", "<", "CA", "CB", ">I", "=I", "<I", ">R", "=R", "<R"], default="<R", space="buy")
    buy_indicator_0 = CategoricalParameter(["trend_ichimoku_base", "trend_ichimoku_a", "trend_kst", "momentum_rsi"], default="trend_ichimoku_base", space="buy")
    buy_cross_0 = CategoricalParameter(["volatility_kcc", "volatility_bbm", "volume_mfi"], default="volatility_kcc", space="buy")
    buy_int_0 = IntParameter(0, 100, default=42, space="buy")
    buy_real_0 = DecimalParameter(0.0, 1.0, default=0.06295, decimals=5, space="buy")
    
    # Hyperopt parameters for sell signals
    sell_oper_0 = CategoricalParameter([">", "=", "<", "CA", "CB", ">I", "=I", "<I", ">R", "=R", "<R"], default="=R", space="sell")
    sell_indicator_0 = CategoricalParameter(["trend_kst_diff", "trend_kst", "momentum_rsi", "trend_macd"], default="trend_kst_diff", space="sell")
    sell_cross_0 = CategoricalParameter(["volume_mfi", "volume_adi", "volume_obv", "volatility_bbm"], default="volume_mfi", space="sell")
    sell_int_0 = IntParameter(0, 100, default=98, space="sell")
    sell_real_0 = DecimalParameter(0.0, 1.0, default=0.8779, decimals=5, space="sell")

    # ROI table:
    minimal_roi = {
        "0": 0.3556,
        "4818": 0.21275,
        "6395": 0.09024,
        "22372": 0
    }

    # Stoploss:
    stoploss = -0.34549

    # Trailing stop:
    trailing_stop = True
    trailing_stop_positive = 0.22673
    trailing_stop_positive_offset = 0.2684
    trailing_only_offset_is_reached = True
    
    timeframe = "12h"

    def populate_indicators(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe = dropna(dataframe)
        dataframe = add_all_ta_features(
            dataframe, open="open", high="high", low="low", close="close", volume="volume",
            fillna=True)
        return dataframe

    def populate_entry_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        conditions = []
        
        # Use hyperopt parameters
        OPR = self.buy_oper_0.value
        IND = self.buy_indicator_0.value
        CRS = self.buy_cross_0.value
        INT = self.buy_int_0.value
        REAL = self.buy_real_0.value
        
        DFIND = dataframe[IND]
        DFCRS = dataframe[CRS]

        if OPR == ">":
            conditions.append(DFIND > DFCRS)
        elif OPR == "=":
            conditions.append(np.isclose(DFIND, DFCRS))
        elif OPR == "<":
            conditions.append(DFIND < DFCRS)
        elif OPR == "CA":
            conditions.append(qtpylib.crossed_above(DFIND, DFCRS))
        elif OPR == "CB":
            conditions.append(qtpylib.crossed_below(DFIND, DFCRS))
        elif OPR == ">I":
            conditions.append(DFIND > INT)
        elif OPR == "=I":
            conditions.append(DFIND == INT)
        elif OPR == "<I":
            conditions.append(DFIND < INT)
        elif OPR == ">R":
            conditions.append(DFIND > REAL)
        elif OPR == "=R":
            conditions.append(np.isclose(DFIND, REAL))
        elif OPR == "<R":
            conditions.append(DFIND < REAL)

        if conditions:
            dataframe.loc[
                reduce(lambda x, y: x & y, conditions),
                "enter_long"] = 1
        return dataframe

    def populate_exit_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        conditions = []
        
        # Use hyperopt parameters
        OPR = self.sell_oper_0.value
        IND = self.sell_indicator_0.value
        CRS = self.sell_cross_0.value
        INT = self.sell_int_0.value
        REAL = self.sell_real_0.value
        
        DFIND = dataframe[IND]
        DFCRS = dataframe[CRS]

        if OPR == ">":
            conditions.append(DFIND > DFCRS)
        elif OPR == "=":
            conditions.append(np.isclose(DFIND, DFCRS))
        elif OPR == "<":
            conditions.append(DFIND < DFCRS)
        elif OPR == "CA":
            conditions.append(qtpylib.crossed_above(DFIND, DFCRS))
        elif OPR == "CB":
            conditions.append(qtpylib.crossed_below(DFIND, DFCRS))
        elif OPR == ">I":
            conditions.append(DFIND > INT)
        elif OPR == "=I":
            conditions.append(DFIND == INT)
        elif OPR == "<I":
            conditions.append(DFIND < INT)
        elif OPR == ">R":
            conditions.append(DFIND > REAL)
        elif OPR == "=R":
            conditions.append(np.isclose(DFIND, REAL))
        elif OPR == "<R":
            conditions.append(DFIND < REAL)

        if conditions:
            dataframe.loc[
                reduce(lambda x, y: x & y, conditions),
                "exit_long"] = 1
        return dataframe
'''

# Create strategy folder
strategy_dir = USER_DATA / 'strategies'
strategy_dir.mkdir(parents=True, exist_ok=True)
strategy_file = strategy_dir / 'GodStra.py'
strategy_file.write_text(strategy_code)

print(f"✓ Strategy created: {strategy_file}")
print(f"  Using binancebot imports (not freqtrade)")
print(f"  Hyperopt parameters: 5 buy + 5 sell = 10 total")
print(f"  Fixed: volatility_kcm → volatility_kcc (correct ta library column name)")

In [ ]:
# CELL 6: Download Data
def download_data(timeframe, timerange):
    """Download historical data"""
    print(f"\n📥 Downloading {timeframe} data ({timerange})...")
    
    cmd = [
        sys.executable, '-m', 'binancebot', 'download-data',
        '--pairs', *CONFIG['pairs'],
        '--timeframe', timeframe,
        '--timerange', timerange,
        '--config', str(CONFIG_PATH),
        '--userdir', str(USER_DATA)
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✓ {timeframe} downloaded for {timerange}")
        # Show last few lines of output for confirmation
        if result.stdout:
            lines = result.stdout.strip().split('\n')
            for line in lines[-3:]:
                if line.strip():
                    print(f"  {line}")
    else:
        print(f"✗ Error downloading {timeframe}")
        print("STDOUT:", result.stdout[-1000:] if result.stdout else "None")
        print("STDERR:", result.stderr[-1000:] if result.stderr else "None")

# Download data for both training and testing periods
print("Downloading data for all timeframes...")
print(f"⚠️  Note: We can only download data up to TODAY ({CONFIG['backtest_timerange'].split('-')[1]})")
print(f"    If download stops early, it means Binance doesn't have more recent data available")
import sys

# Combine train and test periods into one download range
full_range = f"{CONFIG['hyperopt_timerange'].split('-')[0]}-{CONFIG['backtest_timerange'].split('-')[1]}"
print(f"\n📊 Full data range: {full_range}")
print(f"   Training: {CONFIG['hyperopt_timerange']}")
print(f"   Testing:  {CONFIG['backtest_timerange']}\n")

for tf in CONFIG['timeframes']:
    download_data(tf, full_range)

print("\n✓ Data download complete!")
print("\n💡 Tip: Check the output above to see where data actually ends")
print("   If it ends before expected date, that's the most recent data available")

In [ ]:
# CELL 7: Run Hyperopt & Backtest
import sys

def run_hyperopt(timeframe):
    """Run hyperopt for one timeframe"""
    print(f"\n{'='*70}")
    print(f"🔧 HYPEROPT: {CONFIG['strategy']} @ {timeframe}")
    print(f"   Epochs: {CONFIG['epochs']} | Loss: {CONFIG['loss_function']}")
    print(f"{'='*70}\n")
    
    cmd = [
        sys.executable, '-m', 'binancebot', 'hyperopt',
        '--strategy', CONFIG['strategy'],
        '--timeframe', timeframe,
        '--timerange', CONFIG['hyperopt_timerange'],
        '--epochs', str(CONFIG['epochs']),
        '--hyperopt-loss', CONFIG['loss_function'],
        '--config', str(CONFIG_PATH),
        '--userdir', str(USER_DATA),
        '--spaces', 'default',  # Use 'default' to optimize all available spaces
        '--dry-run-wallet', '1000',  # Initial balance for dry-run (avoid exchange API calls)
        '--print-all'  # Show all results
    ]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    epoch_count = 0
    all_output = []
    best_result = None
    
    print("⏳ Starting hyperopt... (this will take ~7 hours)")
    print()
    
    for line in process.stdout:
        all_output.append(line)
        line_stripped = line.rstrip()
        
        # Count epochs
        if 'Best result:' in line or 'New best:' in line:
            epoch_count += 1
            best_result = line_stripped
            # Show progress every epoch with current best
            print(f"✓ Epoch {epoch_count}/{CONFIG['epochs']}: {line_stripped}")
        
        # Show important metrics
        elif 'Objective:' in line:
            print(f"   {line_stripped}")
        
        # Show epoch summary
        elif '/100]' in line or f'/{CONFIG["epochs"]}]' in line:
            print(f"   {line_stripped}")
        
        # Show errors immediately
        elif any(keyword in line.lower() for keyword in ['error', 'exception', 'traceback', 'failed', 'warning']):
            print(f"⚠️  {line_stripped}")
        
        # Show loading/preparation messages
        elif any(keyword in line for keyword in ['Loading data', 'Analyzing', 'Preparing', 'Min roi table']):
            print(f"   {line_stripped}")
    
    process.wait()
    
    print()
    if process.returncode == 0:
        print(f"✅ Hyperopt COMPLETE: {timeframe}")
        if best_result:
            print(f"   {best_result}")
        print(f"   Total epochs completed: {epoch_count}")
        return True
    else:
        print(f"❌ Hyperopt FAILED: {timeframe}")
        print("\n🔍 Last 50 lines of output:")
        print(''.join(all_output[-50:]))
        return False

def run_backtest(timeframe):
    """Run backtest for one timeframe"""
    print(f"\n{'='*70}")
    print(f"📊 BACKTEST: {CONFIG['strategy']} @ {timeframe}")
    print(f"   Timerange: {CONFIG['backtest_timerange']}")
    print(f"{'='*70}\n")
    
    cmd = [
        sys.executable, '-m', 'binancebot', 'backtesting',
        '--strategy', CONFIG['strategy'],
        '--timeframe', timeframe,
        '--timerange', CONFIG['backtest_timerange'],
        '--config', str(CONFIG_PATH),
        '--userdir', str(USER_DATA),
        '--dry-run-wallet', '1000'  # Initial balance for dry-run
    ]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    output_lines = []
    print("⏳ Running backtest...\n")
    
    for line in process.stdout:
        output_lines.append(line)
        line_stripped = line.rstrip()
        
        # Show all important output in real-time
        if any(keyword in line for keyword in [
            'BACKTESTING REPORT', 'SUMMARY METRICS', 'Total/Daily', 
            'Win', 'Loss', 'Avg profit', 'Total profit', 'Trades', 
            'Drawdown', 'Sharpe', 'Sortino', 'Expectancy',
            'Loading data', 'Running backtesting'
        ]):
            print(line_stripped)
        
        # Show errors
        elif any(keyword in line.lower() for keyword in ['error', 'warning', 'failed']):
            print(f"⚠️  {line_stripped}")
    
    process.wait()
    
    print()
    if process.returncode == 0:
        print(f"✅ Backtest COMPLETE: {timeframe}")
        metrics = parse_backtest_output(''.join(output_lines))
        return True, metrics
    else:
        print(f"❌ Backtest FAILED: {timeframe}")
        print("\n🔍 Last 30 lines of output:")
        print(''.join(output_lines[-30:]))
        return False, {}

def parse_backtest_output(output):
    """Parse backtest output to extract metrics"""
    metrics = {}
    for line in output.split('\n'):
        if 'Total profit' in line or 'Absolute profit' in line:
            parts = line.split('|')
            if len(parts) >= 2:
                metrics['total_profit'] = parts[1].strip()
        elif 'Total trades' in line:
            parts = line.split('|')
            if len(parts) >= 2:
                metrics['trades'] = parts[1].strip()
        elif 'Max Drawdown' in line:
            parts = line.split('|')
            if len(parts) >= 2:
                metrics['max_drawdown'] = parts[1].strip()
    return metrics

# Run pipeline for all timeframes
results = {}
all_metrics = []

for idx, tf in enumerate(CONFIG['timeframes'], 1):
    print(f"\n\n{'#'*70}")
    print(f"# COMBINATION {idx}/{len(CONFIG['timeframes'])}: {tf}")
    print(f"{'#'*70}")
    
    # Hyperopt
    hyperopt_success = run_hyperopt(tf)
    
    # Backtest
    backtest_success = False
    metrics = {}
    
    if hyperopt_success:
        backtest_success, metrics = run_backtest(tf)
    
    # Store results
    results[tf] = {
        'hyperopt': 'SUCCESS' if hyperopt_success else 'FAILED',
        'backtest': 'SUCCESS' if backtest_success else 'FAILED',
        'metrics': metrics
    }
    
    if backtest_success:
        all_metrics.append({'timeframe': tf, **metrics})
    
    # Progress summary
    print(f"\n{'='*70}")
    print(f"📊 PROGRESS SUMMARY")
    print(f"{'='*70}")
    for timeframe, data in results.items():
        h_icon = '✅' if data['hyperopt'] == 'SUCCESS' else '❌'
        b_icon = '✅' if data['backtest'] == 'SUCCESS' else '❌'
        print(f"{timeframe:>4s}: Hyperopt {h_icon} | Backtest {b_icon}", end='')
        if data['metrics']:
            profit = data['metrics'].get('total_profit', 'N/A')
            trades = data['metrics'].get('trades', 'N/A')
            print(f" | Profit: {profit} | Trades: {trades}")
        else:
            print()
    print(f"{'='*70}\n")

In [ ]:
# CELL 8: Save Results
output_dir = WORK_DIR / 'training_results'
output_dir.mkdir(exist_ok=True)

# Copy hyperopt results
hyperopt_dir = USER_DATA / 'hyperopt_results'
if hyperopt_dir.exists():
    for file in hyperopt_dir.glob('*.fthypt'):
        shutil.copy(file, output_dir)
        print(f"✓ Saved: {file.name}")

# Copy optimized parameters
strategy_json = USER_DATA / 'strategies' / f'{CONFIG["strategy"]}.json'
if strategy_json.exists():
    shutil.copy(strategy_json, output_dir)
    print(f"✓ Saved: {strategy_json.name}")

# Copy backtest results
backtest_dir = USER_DATA / 'backtest_results'
if backtest_dir.exists():
    for file in backtest_dir.glob('*.json'):
        shutil.copy(file, output_dir)
        print(f"✓ Saved: {file.name}")

# Create summary file
summary_file = output_dir / 'complete_summary.txt'
with open(summary_file, 'w', encoding='utf-8') as f:
    f.write(f"Complete Training & Backtest Summary\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"{'='*70}\n\n")
    f.write(f"Strategy: {CONFIG['strategy']}\n")
    f.write(f"Epochs: {CONFIG['epochs']}\n")
    f.write(f"Timeframes: {', '.join(CONFIG['timeframes'])}\n")
    f.write(f"Train Period: {CONFIG['hyperopt_timerange']}\n")
    f.write(f"Test Period: {CONFIG['backtest_timerange']}\n\n")
    
    f.write("="*70 + "\n")
    f.write("RESULTS BY TIMEFRAME\n")
    f.write("="*70 + "\n\n")
    
    for tf, data in results.items():
        f.write(f"\nTimeframe: {tf}\n")
        f.write(f"  Hyperopt: {data['hyperopt']}\n")
        f.write(f"  Backtest: {data['backtest']}\n")
        
        if data['metrics']:
            f.write(f"  Metrics:\n")
            for key, value in data['metrics'].items():
                f.write(f"    {key}: {value}\n")
        f.write("\n")

print(f"\n✓ Results saved to: {output_dir}")

# Export metrics to CSV
if all_metrics:
    import csv
    csv_file = output_dir / 'backtest_metrics.csv'
    
    all_keys = set()
    for m in all_metrics:
        all_keys.update(m.keys())
    
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=sorted(all_keys))
        writer.writeheader()
        writer.writerows(all_metrics)
    
    print(f"✓ Metrics CSV saved: {csv_file}")

print(f"\n📥 Download 'training_results' folder from Output tab!")

In [ ]:
# CELL 9: Display Summary
print("\n" + "="*70)
print("🎉 TRAINING & BACKTEST COMPLETE!")
print("="*70)

print(f"\nStrategy: {CONFIG['strategy']}")
print(f"Epochs per timeframe: {CONFIG['epochs']}")
print(f"Total timeframes: {len(CONFIG['timeframes'])}")

print(f"\n📊 RESULTS SUMMARY:")
print("-"*70)

for tf, data in results.items():
    h_status = '✅' if data['hyperopt'] == 'SUCCESS' else '❌'
    b_status = '✅' if data['backtest'] == 'SUCCESS' else '❌'
    
    print(f"\n{tf}:")
    print(f"  Hyperopt: {h_status}")
    print(f"  Backtest: {b_status}")
    
    if data['metrics']:
        print(f"  Metrics:")
        for key, value in data['metrics'].items():
            print(f"    - {key}: {value}")

print("\n" + "="*70)
print(f"📂 Output folder: {output_dir}")
print("="*70)
print("\n📥 Download from Kaggle Output tab:")
print("  - *.fthypt files (hyperopt history)")
print("  - *.json files (backtest results + optimized params)")
print("  - complete_summary.txt (full summary)")
print("  - backtest_metrics.csv (metrics table)")
print("\n💡 Use these files locally:")
print("  1. Copy *.fthypt to user_data/hyperopt_results/")
print("  2. Copy GodStra.json to user_data/strategies/")
print("  3. Run backtest locally with optimized params")